In [44]:
# import os
# import csv
# from pydriller import RepositoryMining
# from radon.raw import analyze
# from radon.metrics import h_visit
# from radon.metrics import h_visit_ast
# from radon.complexity import sorted_results
# from pydriller.git_repository import GitRepository

# fields = ['CommitID','filename','loc','lloc','sloc','comments','multi','blank','code_comment','h1',
#          'h2','N1','N2','vocabulary','length','calculated_length','volume',                     
#          'difficulty','effort','time','bugs']
# releases = ['4.6.6','4.6.7','4.6.8','4.6.9','5.3.0','5.3.1','5.3.2','5.3.3','5.3.4','5.3.5']
# repo = "/Users/sanika/Downloads/Master Thesis/repo_updated/pytest"

# # count = 0
# i = 0
# length = len(releases)-1
# while i < length:
#     for commit in RepositoryMining(repo,
#                                    only_modifications_with_file_types=['.py'],
#                                    from_tag=releases[i],
#                                    to_tag=releases[i+1]).traverse_commits():
#         for modification in commit.modifications:
# #             date = commit.committer_date
#             mod_file = modification.filename
#             hash_val = commit.hash
#             i += 1
#             with open('extracted_data_000.csv', 'w') as csvFile:
#                 writer = csv.DictWriter(csvFile, fieldnames = fields)
#                 writer.writeheader()
#                 for filename in mod_file: #files:
#                     if filename.endswith(".py"):
#                         print(filename)
#                         with open(filename) as f:
#                             content = f.read()
#                             file_analyze = analyze(content) #raw metrics
#                             code_n_comment = file_analyze.loc+file_analyze.comments
#                             file_ast = h_visit(content) #Compile the code into an AST tree
#                             count = count + 1
#                             data = [{'CommitID':(hash_val),
#                                      'filename':(mod_file),
#                                      'loc':(file_analyze.loc),
#                                      'lloc':(file_analyze.lloc),
#                                      'sloc':(file_analyze.sloc),
#                                      'comments':(file_analyze.comments),
#                                      'multi':(file_analyze.multi),
#                                      'blank':(file_analyze.blank),
#                                      'code_comment':(code_n_comment),
#                                      'h1':(file_ast.total.h1),
#                                      'h2':(file_ast.total.h2),
#                                      'N1':(file_ast.total.N1),
#                                      'N2':(file_ast.total.N2),
#                                      'vocabulary':(file_ast.total.vocabulary),
#                                      'length':(file_ast.total.length),
#                                      'calculated_length':(file_ast.total.calculated_length),
#                                      'volume':(file_ast.total.volume),                     
#                                      'difficulty':(file_ast.total.difficulty),
#                                      'effort':(file_ast.total.effort),
#                                      'time':(file_ast.total.time),
#                                      'bugs':(file_ast.total.bugs),}]
#                             writer.writerows(data)

# print("writing completed")
# csvFile.close()

In [2]:
#extract data with release filter
import os
import csv
from pydriller import RepositoryMining
from radon.raw import analyze
from radon.metrics import h_visit
from radon.metrics import h_visit_ast
from radon.complexity import sorted_results
from pydriller.git_repository import GitRepository

fields = ['CommitID','filename','token_count','change_type','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs']
# 'change_type','lines_added','lines_removed',
releases = ['4.6.6','4.6.7','4.6.8','4.6.9','5.3.0','5.3.1','5.3.2','5.3.3','5.3.4','5.3.5']
repo_updated = "/Users/sanika/Downloads/Master Thesis/repo_updated/pytest"
    
# gitrepo = GitRepository("/Users/sanika/Downloads/Master Thesis/repo_updated/pytest")
# gitfiles = gitrepo.files()
# no_commits = gitrepo.total_commits() #total number of commits

count = 0
i = 0
length = len(releases)-1

with open('extracted_data_000.csv', 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames = fields)
    writer.writeheader()
    while i < length:
        from_tag=releases[i]
        to_tag=releases[i+1]
        i += 1
        for commit in RepositoryMining(repo_updated,
                                   only_modifications_with_file_types=['.py'],
                                   from_tag=from_tag,
                                   to_tag=to_tag).traverse_commits():
            for modification in commit.modifications:
                filename = modification.filename
                hash_val = commit.hash
                change_type = modification.change_type
                token_count = modification.token_count
#                 added = modification.added
#                 removed = modification.removed
                if filename.endswith(".py"):
                    for r, d, f in os.walk(repo_updated):
                        for file in f:
                            if filename in file:
                                file_path = os.path.join(r, file)
                                with open(file_path) as f:
                                    content = f.read()
                                    file_analyze = analyze(content)
                                    code_n_comment = file_analyze.loc+file_analyze.comments
                                    file_ast = h_visit(content)
                                    data = [{'CommitID':(hash_val), #put to a dataframe
                                     'filename':(file_path),
                                     'token_count':(token_count),
                                     'change_type':(change_type),
#                                      'lines_added':(added),
#                                      'lines_removed':(removed),
                                     'loc':(file_analyze.loc),
                                     'lloc':(file_analyze.lloc),
                                     'sloc':(file_analyze.sloc),
                                     'commeants':(file_analyze.comments),
                                     'multi':(file_analyze.multi),
                                     'blank':(file_analyze.blank),
                                     'code_comment':(code_n_comment),
                                     'h1':(file_ast.total.h1),
                                     'h2':(file_ast.total.h2),
                                     'N1':(file_ast.total.N1),
                                     'N2':(file_ast.total.N2),
                                     'vocabulary':(file_ast.total.vocabulary),
                                     'length':(file_ast.total.length),
                                     'calculated_length':(file_ast.total.calculated_length),
                                     'volume':(file_ast.total.volume),                     
                                     'difficulty':(file_ast.total.difficulty),
                                     'effort':(file_ast.total.effort),
                                     'time':(file_ast.total.time),
                                     'bugs':(file_ast.total.bugs),}]
                                    writer.writerows(data) # saving the dataframe => df.to_csv('file1.csv') 
                                    i += 1
print("writing completed")
csvFile.close()

writing completed
